In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import os
import re
import matplotlib.pyplot as plt

tqdm.pandas()

In [ ]:
## Run the following to compute the mappings for host locations
#!python3 -m scripts.mappings_host_location_all_cities

## Load Visualization Pipeline
#!python3 -m scripts.pipeline_all_cities_viz

## Load ML pipeline
#!python3 -m scripts.pipeline_ML

In [ ]:
df = pd.read_pickle("data/pickles/total_listings_viz.pkl")

In [ ]:
df.columns.tolist()

## Visualize neighbourhoods

In [ ]:
neigh_list = df["neighbourhood_cleansed"].tolist()
neigh_counter = {x:neigh_list.count(x) for x in neigh_list}
neigh_counter = dict(sorted(neigh_counter.items(), key=lambda item: item[1], reverse=True))

neigh_counter


In [ ]:
df = pd.read_pickle("data/pickles/total_listings_exploration_handling.pkl")

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
len((df["neighbourhood_cleansed"]+"-"+df["listing_city"]).unique().tolist())

In [ ]:
neigh_list = df["listing_city"].tolist()
neigh_counter = {x:neigh_list.count(x) for x in neigh_list}
neigh_counter = dict(sorted(neigh_counter.items(), key=lambda item: item[1], reverse=True))

neigh_counter